In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyphi
import networkx as nx
import os
import seaborn as sns
import pandas as pd
import time
import random
import pickle
import time
import itertools
from scipy.stats import pearsonr

%matplotlib inline


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [2]:
df = pd.DataFrame()

#3
nodes = ('A', 'B', 'C')
#4
#nodes = ('A', 'B', 'C', 'D')
#5
#nodes = ['A', 'B', 'C', 'D', 'E']
#6
#nodes = ['A', 'B', 'C', 'D', 'E', 'F']
#7
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
#8
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
#9
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
#10
#nodes = ['A', 'B', 'C', 'D', 'E', 'F','G', 'H', 'I', 'J']
#11
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K']
n = len(nodes)
networks = 30

In [8]:
def make_cm(nodes):
    n = len(nodes)
    x = n**2-n
    y = 2**(x)
    seed = np.random.randint(1,y+1,dtype = 'int64')
    binary = bin(seed)
    

    d = []
    for i in range(2,len(binary)):
        d.append(int(binary[i]))

    while len(d) != x:
        d.insert(0,0)

    cm_list = []
    cm = []
    
    zeros = np.zeros((n,n), dtype = int)
    s = 0
    for j in range(n):
        for k in range(n):
            if j != k:
                zeros[j][k] = d[s]
                s +=1

    x = []
    axis0 = zeros.sum(axis=0)
    axis1 = zeros.sum(axis=1)
    for l in range(n):
        if axis0[l] == 0 and axis1[l] == 0:       
            x.append(l)
    
            
    if len(x) == 0:
        drop_list = 0
        cm = zeros
    else:
        drop_list = zeros
        cm = 0

        
    return drop_list, cm

In [9]:
drop_list, cm = make_cm(nodes)

In [11]:
while len(df) < networks:
    drop_list, cm = make_cm(nodes)
    if type(cm) != int:
        d = cm .reshape(1,n**2)
        v = d[0]
        v = v.tolist()
        vc = pd.Series(v)
        df = df.append(vc, ignore_index=True)
    df = df.drop_duplicates()

In [13]:
cm_list = []
for i in range(len(df)):
    cm_list.append(df.values[i].reshape(n,n).astype(int))

In [15]:
edge = []
edges = {}
for i in range(len(df)):    
        a = cm_list[i]
        for col in range(len(nodes)):
            for row in range(len(nodes)):
                if a[col][row] == 1:
                    edge.append((nodes[col],nodes[row]))
        edges[i] = edge
        edge = []

In [16]:
cm_path = "./cm/{0}_nodes_cm.npy".format(len(nodes))
edge_path = "./edge/{0}_nodes_edge.json".format(len(nodes))

In [ ]:
np.save(cm_path, cm_list)
with open(edge_path, 'wb') as fp:
    pickle.dump(edges, fp)

In [ ]:
cm_list = np.load(cm_path)
with open(edge_path, 'rb') as fp:
    edges = pickle.load(fp)

In [ ]:
states = np.zeros((pow(2, len(nodes)),len(nodes)), dtype=int)
for i in range(pow(2, len(nodes))):
    for j in range(1, len(bin(i))-1):
        states[i][j-1] = bin(i)[j * (-1)]

In [18]:
success_dic = {}
success_list = []
phis_path = "./phis/{0}_nodes_phis.json".format(len(nodes))
success_state_path = "./success_states/{0}_nodes_success_state.json".format(len(nodes))

In [ ]:
phis = {}
phi = []
for i in range(len(cm_list)):
    tpm = np.dot(states, cm_list[i])  
    s = np.sum(tpm, axis=1)
    a = []
    for j in range(tpm.shape[0] ):
        if s[j] != 0:
            a.append((tpm[j] / s[j]))
        else:
            a.append(tpm[j])
    s_tpm = np.array(a)
    print(i,"cm")

    
    
    
    network = pyphi.Network(s_tpm, cm=cm_list[i], node_labels=nodes)
    for j in range(pow(2, len(nodes))):
        init_state = states[j]
        try:
            print(j, "states")
            subsystem = pyphi.Subsystem(network, init_state)
            p = pyphi.compute.phi(subsystem)
            phi.append(p)
 
            success_list.append(j)

        except:
            
            pass
    phis[i] = phi
    success_dic[i] = success_list

    
    
    success_list = []
    phi = []
    
with open(phis_path, 'wb') as fp:
    pickle.dump(phis, fp)
    
with open(success_state_path, 'wb') as fp:
    pickle.dump(success_dic, fp)